In [ ]:
!pip install lightgbm numerox

import numerox as nx
from os.path import exists
from time import time
import numpy as np
import lightgbm as lgbm
from xgboost import XGBClassifier
import numerapi 
from time import sleep
import numerapi

stake = False
confidence= 0.501
stake_per_tournament=0.1

public_id= "public_id"
secret_key= "secret_key"


if(not exists('numerai_datasets.zip')):
  nx.download('numerai_datasets.zip') 
  
#data = nx.play_data()
data = nx.load_zip('numerai_datasets.zip')

class lightgbm(nx.Model):

    def __init__(self,task='train', verbose=0):
        self.p= {
                'task': task,
                'verbose': verbose
                 }

    def fit_predict(self, dfit, dpre, tournament):
        lgb_train = lgbm.Dataset(dfit.x, dfit.y[tournament])
        model = lgbm.train(self.p,
                    lgb_train)
        yhat = model.predict(dpre.x, num_iteration=model.best_iteration)
        yhat[yhat<0.3]=0.3
        yhat[yhat>0.7]=0.7
        return dpre.ids, yhat


class xgb(nx.Model):
    def __init__(self, max_depth=2):
        self.p={
        'max_depth':max_depth
        }


    def fit_predict(self, dfit, dpre, tournament):
        model = XGBClassifier(max_depth=self.p['max_depth'], silent= False, n_jobs=-1)
        model.fit(dfit.x, dfit.y[tournament])
        yhat = model.predict_proba(dpre.x)[:,1]
        yhat[yhat<0.3]=0.3
        yhat[yhat>0.7]=0.7
        return dpre.ids, yhat


#Training
model = lightgbm(task='train', verbose=0) #xgb(max_depth=2)
splitter= nx.TournamentSplitter(data)
prediction = nx.run(model, splitter)


#Submission
for tournament in nx.tournament_all():
  path = tournament+'_submission.csv'
  prediction[:, tournament].to_csv(path)
  try:
    upload_id, status = nx.upload(path, tournament, public_id, secret_key)
    sleep(60)
  except:
    pass
print("User submitted!")

#Stacking 
if stake: 
  napi = numerapi.NumerAPI(public_id, secret_key)
  for j in range(1,5):
    napi.stake(confidence, stake_per_tournament, tournament=j)
  print('Account staked!')
